In [ ]:
%pylab inline
%matplotlib inline

import numpy as np
import cv2
import time
import TreeData as td
import pickle
import ImageProcessing as ipr
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
from sklearn import tree
import BinaryTree as bt
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydot
import graphviz as gz

In [ ]:
pickleFile  = open('my data/' + 'rules 3 pixels' + '.pkl', 'rb')
optiRules = pickle.load(pickleFile)
pickleFile.close()

baseRule = lambda x: (x.any() == x.any())

fig = figure(0, (5 * len(optiRules), 10 * len(optiRules)))
for i in range(len(optiRules)):
    indices = optiRules[i][0]
    rule = bt.createRuleFromIndices(indices, baseRule)
    optiRules[i].insert(0, rule)
    subplot(1, len(optiRules), i+1)
    box = bt.visualiseRule(indices)
    title(optiRules[i][1])
    imshow(box, cmap='seismic')

print('Correctly opened ' + str(len(optiRules)) + ' optimal rule(s) from a tree !')

In [ ]:
cap = cv2.VideoCapture(0)
ret = cap.read()

print('Press \'q\' to quit the camera')

while(ret):
    #Capture frame-by-frame
    
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    
    cutImage, coloredFrame, dimensions = ipr.reduce_image(frame, 195, 135, True)
    resizedImage = cv2.resize(cutImage, (45, 65)) 
    smallGray = cv2.cvtColor(resizedImage, cv2.COLOR_BGR2GRAY)
    
    matches, num = ipr.live_matches(smallGray, resizedImage, 110,\
                                    optiRules, [0, 0, 150])
        
    bigMatches = cv2.resize(matches, (135, 195))
    bigGray = cv2.resize(smallGray, (135, 195))
    #coloredImage = ipr.color_square(cutImage)
    
    insertedFrame = ipr.insert_image(coloredFrame.copy(), bigMatches, dimensions)
    
    cv2.putText(insertedFrame, str(num) + ' fingers', (15, 50),\
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), lineType=cv2.LINE_AA) 
    
    cv2.imshow('webcam image', insertedFrame)
    #cv2.imshow('matches', bigMatches)
    #cv2.imshow('gray scale box', bigGray)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print('Camera Closed')
        break    

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()